In [1]:
from typing import Optional

# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel

import nest_asyncio
from pyngrok import ngrok
import uvicorn
import json

In [2]:
with open('account.json', 'r') as fp:
    account = json.load(fp)
print(account)

{'abc': '1234', 'bcd': '2345', 'cdef': '3456'}


In [3]:
with open('pet_num.json', 'r') as fp:
    data = json.load(fp)
print(data)

{'abc': 20, 'bcd': 5, 'cdef': 35}


In [4]:
def pet_sum(id:str, pet_num:int):
  if id in list(data.keys()):
    data[id]+= pet_num
    return data[id]
  else:
    new_m_p = {id : pet_num}
    data.update(new_m_p)
    return data[id]

In [5]:
from fastapi import FastAPI
from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from core.config import settings
from apis.general_pages.route_homepage import general_pages_router
from fastapi.responses import HTMLResponse

templates = Jinja2Templates(directory="templates")

def include_router(app):
    app.include_router(general_pages_router)


def configure_static(app):  #new
    app.mount("/static", StaticFiles(directory="static"), name="static")


def start_application():
    app = FastAPI(title=settings.PROJECT_NAME,version=settings.PROJECT_VERSION)
    include_router(app)
    configure_static(app) #new
    return app

app = start_application()

# 웹페이지만 나옴 
@app.get('/login/')
def login(request: Request):
    return templates.TemplateResponse("auth/login.html", {"request": request})

# 로그인 결과 출력 
@app.get('/login/result')
def info(request: Request, id:str, pw:str):
    if id in account.keys():
        if account[id] == pw:
            print('pass')
            return {f'{id}님 성공적으로 로그인하셨습니다.'}
        else:
            print('fail')
            return{'비밀번호를 잘못 입력하셨습니다. 다시 로그인해주세요.'}
    else:
      return {'회원가입 후 시도해주세요.'}

# 페트병 개수 
@app.get('/pet/', response_class=HTMLResponse)
async def pet(request: Request, user_id:str, pet_num:int):
    result = pet_sum(user_id, pet_num)
    return templates.TemplateResponse("main.html", {"request": request, 'id':user_id, 'pet_num':result})

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [6714]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://3bf7-114-203-149-236.ngrok.io
INFO:     127.0.0.1:55648 - "GET /pet/?user_id=abc&pet_num=10 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55649 - "GET /pet/?user_id=abc&pet_num=20 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55649 - "GET /static/images/nolabel.png HTTP/1.1" 304 Not Modified


# 미 해결과제 (HELP)

1. 데이터 업로드(사용자, 패트병 수)된 내용 json으로 다시 저장 및 업데이트 하기 
    - 참고 : fastapi로 작동해서 데이터는 올라가는데, 파일 형식으로 다시 저장해야할듯 
2. qr코드 인식은 되지만 한 프로세스로 연결할 필요가 있어보임! 